# Reading in the data

In [2]:
import pandas as pd
import sqlite3

academy_awards = pd.read_csv("datasets/academy_awards.csv", encoding = "ISO-8859-1")
#print(academy_awards.head(5))

# Cleaning the data

In [3]:
# Cleaning Year column 
# Year column contains information of version of academy award
# as well. We need to extract the first 4 digits from Year col
academy_awards['Year'] = academy_awards['Year'].str[0:4]
academy_awards['Year'] = academy_awards['Year'].astype('int64')

In [4]:
# Extract movies latter 2000
latter_than_2000 = academy_awards[academy_awards['Year'] > 2000]

# We are interested in only following award categories
award_categories = ['Actor -- Leading Role', 'Actor -- Supporting Role',
                   'Actress -- Leading Role', 'Actress -- Supporting Role']

nominations = latter_than_2000[latter_than_2000['Category'].isin(award_categories)]

nominations.head()

,Year,Category,Nominee,Additional Info,Won?,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,2010,Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},NO,NaN,NaN,NaN,NaN,NaN,NaN
1,2010,Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},NO,NaN,NaN,NaN,NaN,NaN,NaN
2,2010,Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},NO,NaN,NaN,NaN,NaN,NaN,NaN
3,2010,Actor -- Leading Role,Colin Firth,The King's Speech {'King George VI'},YES,NaN,NaN,NaN,NaN,NaN,NaN
4,2010,Actor -- Leading Role,James Franco,127 Hours {'Aron Ralston'},NO,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Cleaning 'Won?' columns
map_dict = {'YES': 1, 'NO': 0}
nominations['Won?'] = nominations['Won?'].map(map_dict)
nominations['Won'] = nominations['Won?']

cols_for_dropping = ['Won?', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7',
                   'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10']

final_nominations = nominations.drop(cols_for_dropping, axis = 1)
final_nominations.head(5)

/Users/yogeshsoniwal/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/yogeshsoniwal/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Year,Category,Nominee,Additional Info,Won
0,2010,Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},0
1,2010,Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},0
2,2010,Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},0
3,2010,Actor -- Leading Role,Colin Firth,The King's Speech {'King George VI'},1
4,2010,Actor -- Leading Role,James Franco,127 Hours {'Aron Ralston'},0


# Getting information from Additional Info column

In [6]:
additional_info_one = final_nominations['Additional Info'].apply(lambda x: x.rstrip("'}"))
additional_info_two = additional_info_one.apply(lambda x: x.split(" {'"))

movie_names = additional_info_two.apply(lambda x: x[0])
characters = additional_info_two.apply(lambda x: x[1])

final_nominations['Movie'] = movie_names
final_nominations['Character'] = characters

final_nominations.head(5)

,Year,Category,Nominee,Additional Info,Won,Movie,Character
0,2010,Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},0,Biutiful,Uxbal
1,2010,Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},0,True Grit,Rooster Cogburn
2,2010,Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},0,The Social Network,Mark Zuckerberg
3,2010,Actor -- Leading Role,Colin Firth,The King's Speech {'King George VI'},1,The King's Speech,King George VI
4,2010,Actor -- Leading Role,James Franco,127 Hours {'Aron Ralston'},0,127 Hours,Aron Ralston


# Export the data for SQLite

In [7]:
# Since the database with this name doesn't exist, it 
# will be created
conn = sqlite3.connect('nominations.db')

# Export the nominations DataFrame to the db
# DF will be exported to 'nominations' table
final_nominations.to_sql('nominations', conn, index = False)

conn.close()

/Users/yogeshsoniwal/anaconda/lib/python2.7/site-packages/pandas/core/generic.py:1362: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


# Verifying the SQL export

In [8]:
conn = sqlite3.connect('nominations.db')

# Get schema
query = "PRAGMA table_info(nominations);"
schema = conn.execute(query).fetchall()
print(schema)

# Get first 10 rows
query = "SELECT * FROM nominations LIMIT 10;"
first_10_rows = conn.execute(query).fetchall()
print(first_10_rows)

conn.close()

[(0, u'Year', u'INTEGER', 0, None, 0), (1, u'Category', u'TEXT', 0, None, 0), (2, u'Nominee', u'TEXT', 0, None, 0), (3, u'Additional Info', u'TEXT', 0, None, 0), (4, u'Won', u'INTEGER', 0, None, 0), (5, u'Movie', u'TEXT', 0, None, 0), (6, u'Character', u'TEXT', 0, None, 0)]
[(2010, u'Actor -- Leading Role', u'Javier Bardem', u"Biutiful {'Uxbal'}", 0, u'Biutiful', u'Uxbal'), (2010, u'Actor -- Leading Role', u'Jeff Bridges', u"True Grit {'Rooster Cogburn'}", 0, u'True Grit', u'Rooster Cogburn'), (2010, u'Actor -- Leading Role', u'Jesse Eisenberg', u"The Social Network {'Mark Zuckerberg'}", 0, u'The Social Network', u'Mark Zuckerberg'), (2010, u'Actor -- Leading Role', u'Colin Firth', u"The King's Speech {'King George VI'}", 1, u"The King's Speech", u'King George VI'), (2010, u'Actor -- Leading Role', u'James Franco', u"127 Hours {'Aron Ralston'}", 0, u'127 Hours', u'Aron Ralston'), (2010, u'Actor -- Supporting Role', u'Christian Bale', u"The Fighter {'Dicky Eklund'}", 1, u'The Fighter', 